In [1]:
from IPython.display import HTML

# Load the IBM Plex Mono font
HTML('''<style>
@import url('https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@400;700&display=swap');
</style>''')

In [2]:
import os
import pandas as pd

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo
import plotly.colors as pc

import datetime as dt
from datetime import timedelta
import sys
import requests
from dotenv import load_dotenv

from web3 import Web3
from eth_abi import decode
from eth_utils import decode_hex, to_text
from web3.middleware import geth_poa_middleware
import time
import json
from dash import Dash, html, dcc, Input, Output, State, callback
from dash import dash_table

import requests
from dune_client.client import DuneClient

from IPython.display import Image, display

pio.templates["custom"] = pio.templates["plotly"]
pio.templates["custom"].layout.font.family = "IBM Plex Mono"

# Set the default template
pio.templates.default = "custom"

In [3]:
from chart_builder.scripts.visualization_pipeline import visualization_pipeline
from chart_builder.scripts.utils import data_processing,create_df,open_json, main as chartBuilder

sys.path: ['C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311', 'e:\\Projects\\rlusd_dash\\rlusd_env', '', 'e:\\Projects\\rlusd_dash\\rlusd_env\\Lib\\site-packages', 'e:\\projects\\chart_builder_pkg', 'e:\\Projects\\rlusd_dash\\rlusd_env\\Lib\\site-packages\\win32', 'e:\\Projects\\rlusd_dash\\rlusd_env\\Lib\\site-packages\\win32\\lib', 'e:\\Projects\\rlusd_dash\\rlusd_env\\Lib\\site-packages\\Pythonwin', 'e:\\projects\\chart_builder_pkg\\chart_builder\\scripts']
Current working directory: e:\Projects\rlusd_dash\notebooks
Current directory: e:\projects\chart_builder_pkg\chart_builder\scripts
Current working directory: e:\Projects\rlusd_dash\notebooks
Current directory: e:\projects\chart_builder_pkg\chart_builder\scripts


In [4]:
os.chdir('..')

In [5]:
from sql_queries.sql_scripts import lp_data
from python_scripts.utils import flipside_api_results

In [6]:
print(os.getcwd())

e:\Projects\rlusd_dash


In [7]:
load_dotenv()

True

In [8]:
import random

def colors(shuffle=False):
    on_colors = ['#00CC96', '#67D8FE','#FF7EBE', '#FFBC36', '#E6FF59']
    
    color_palette = pc.qualitative.Plotly[::-1]
    distinct_palette = pc.qualitative.Dark24 + pc.qualitative.Set3
    lib_colors = distinct_palette+color_palette

    if shuffle==True:
        random.shuffle(lib_colors)

    combined_colors = on_colors + lib_colors
    print(f'Combined colors: {combined_colors} \nCombined colors length: {len(combined_colors)}')
    
    return combined_colors

combined_colors = colors()
font_family = 'IBM Plex Mono'

Combined colors: ['#00CC96', '#67D8FE', '#FF7EBE', '#FFBC36', '#E6FF59', '#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#DA16FF', '#222A2A', '#B68100', '#750D86', '#EB663B', '#511CFB', '#00A08B', '#FB00D1', '#FC0080', '#B2828D', '#6C7C32', '#778AAE', '#862A16', '#A777F1', '#620042', '#1616A7', '#DA60CA', '#6C4516', '#0D2A63', '#AF0038', 'rgb(141,211,199)', 'rgb(255,255,179)', 'rgb(190,186,218)', 'rgb(251,128,114)', 'rgb(128,177,211)', 'rgb(253,180,98)', 'rgb(179,222,105)', 'rgb(252,205,229)', 'rgb(217,217,217)', 'rgb(188,128,189)', 'rgb(204,235,197)', 'rgb(255,237,111)', '#FECB52', '#FF97FF', '#B6E880', '#FF6692', '#19D3F3', '#FFA15A', '#AB63FA', '#00CC96', '#EF553B', '#636EFA'] 
Combined colors length: 51


In [9]:
def dune_api_results(query_num, save_csv=False, csv_path=None):
    results = dune.get_latest_result(query_num)
    df = pd.DataFrame(results.result.rows)

    if save_csv and csv_path:
        df.to_csv(csv_path, index=False)
    return df

In [10]:
RLUSD_ETHEREUM = os.getenv('RLUSD_ETHEREUM')
RLUSD_XRP = os.getenv('RLUSD_XRP')
DUNE_KEY = os.getenv('DUNE_API_KEY')
FLIPSIDE_KEY = os.getenv('FLIPSIDE_API_KEY')

In [11]:
dune = DuneClient(DUNE_KEY)

In [12]:
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5'))

In [13]:
erc20_abi_path = 'abi/erc20_abi.json'

abi_paths = [erc20_abi_path]
abis = {}

for path in abi_paths:
    with open(path, "r") as file:
        abis[path] = json.load(file)

print(abis)

{'abi/erc20_abi.json': [{'constant': True, 'inputs': [], 'name': 'name', 'outputs': [{'name': '', 'type': 'string'}], 'payable': False, 'stateMutability': 'view', 'type': 'function'}, {'constant': False, 'inputs': [{'name': '_spender', 'type': 'address'}, {'name': '_value', 'type': 'uint256'}], 'name': 'approve', 'outputs': [{'name': '', 'type': 'bool'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'constant': True, 'inputs': [], 'name': 'totalSupply', 'outputs': [{'name': '', 'type': 'uint256'}], 'payable': False, 'stateMutability': 'view', 'type': 'function'}, {'constant': False, 'inputs': [{'name': '_from', 'type': 'address'}, {'name': '_to', 'type': 'address'}, {'name': '_value', 'type': 'uint256'}], 'name': 'transferFrom', 'outputs': [{'name': '', 'type': 'bool'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'constant': True, 'inputs': [], 'name': 'decimals', 'outputs': [{'name': '', 'type': 'uint8'}], 'payable': False, 'st

In [14]:
rlusd_contract = w3.eth.contract(address=RLUSD_ETHEREUM, abi=abis['abi/erc20_abi.json'])

In [15]:
rlusd_ETH_supply = rlusd_contract.functions.totalSupply().call() /1e18
rlusd_ETH_supply

85337632.55

In [16]:
def call_api(base_url):
    response = requests.get(base_url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        print("data:", data)
        return data
    else:
        print("Error:", response.status_code, response.text)
        return None
    

In [17]:
base_url = f'https://api.xrpscan.com/api/v1/account/{RLUSD_XRP}/obligations'
data = call_api(base_url)

data: [{'currency': '524C555344000000000000000000000000000000', 'value': '37479104.2900001'}]


In [18]:
rlusd_raw = pd.DataFrame(data)
rlusd_XRP_supply = float(rlusd_raw['value'].values[0])

In [19]:
rlusd_XRP_supply

37479104.2900001

In [20]:
supply_dict = {
    "Blockchain":["Ethereum","XRP"],
    "Supply":[rlusd_ETH_supply,rlusd_XRP_supply]
}

supply_dict

{'Blockchain': ['Ethereum', 'XRP'], 'Supply': [85337632.55, 37479104.2900001]}

In [21]:
supply_df = pd.DataFrame(supply_dict)

In [22]:
supply_df.to_csv('data/rl_usd_supply.csv')

In [23]:
total_rlusd_supply = supply_df['Supply'].sum()
total_rlusd_supply

np.float64(122816736.8400001)

In [24]:
fig1 = visualization_pipeline(
df=supply_df,
    title='rlusd_fig1',
    # start_date='2024-01-01',
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='pie',
    dimensions=dict(height=351, width=730),
    groupby = 'Blockchain',
    num_col='Supply',
    # cols_to_plot='All',
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['UAW'],y2=[]),
    tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=0,
    y2=True,
    # min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines+markers',
    # font_size=14,
    # line_width=3,
    margin=dict(l=0, r=0, t=50, b=0),
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=False,
    hole_size=0,
    line_width=0,
    descending=False,
    buffer = 0.25,
    annotations=True,
    text=True,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=True,
    annotation_prefix=None,
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['Ethereum' 'XRP']


In [25]:
chartBuilder(
    fig=fig1,
    add_the_date=False,
    clean_columns=True,
    date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    save=False,
    topn=4,
    title_xy=dict(x=0.1,y=0.92),
    title='RLUSD Supply by Chain',
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
capwords: []
y1_list: All 
y2_list: []
'Blockchain' does not match capword. Converting to title case: Blockchain
Cleaned string: 'Blockchain'
'Supply' does not match capword. Converting to title case: Supply
Cleaned string: 'Supply'
'A' does not match capword. Converting to title case: A
Cleaned string: 'A'
'l' does not match capword. Converting to title case: L
Cleaned string: 'L'
'l' does not match capword. Converting to title case: L
Cleaned string: 'L'
'Blockchain' does not match capword. Converting to title case: Blockchain
Cleaned string: 'Blockchain'
Cleaned groupby: Blockchain
'Supply' does not match capword. Converting to title case: Supply
Cleaned string: 'Supply'
Cleaned num_col: Supply
Cleaned DataFrame columns: ['Blockchain', 'Supply']
Cleaned cols_to_plot: ['Blockchain', 'Supply']
Cleaned bar_col: []
Cleaned line_col: []
Cleaned y1_list: ['A', 'L', 'L']
Cleaned y2_list: []
Cleaned groupby: Blockchain
Cleaned num_col: Supply
logo path: ../img/Logos/None
Generati

In [26]:
def get_pagination_results(base_url):
    # Pagination parameters
    limit = 100  # Number of records per request
    offset = 0  # Start from the first record
    all_results = []  # Store all retrieved results

    while True:
        # Construct URL with pagination parameters
        url = f"{base_url}?offset={offset}&limit={limit}"
        
        # Make API request
        response = requests.get(url)
        
        # Check for successful response
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break
        
        # Parse JSON response
        data = response.json()
        
        # Append results
        if not data:  # Stop when no more data is returned
            break
        
        all_results.extend(data)
        
        # Update offset to fetch the next batch
        offset += limit

    # Print the total number of results retrieved
    print(f"Total records fetched: {len(all_results)}")
    return all_results

In [27]:
def get_tx_pagination_results(base_url):
    all_results = []  # Store all retrieved results
    marker = None     # Marker for pagination

    while True:
        # Construct URL with pagination parameters
        if marker:
            url = f"{base_url}?marker={marker}"
        else:
            url = base_url
        
        # Make API request
        response = requests.get(url)
        
        # Check for successful response
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break
        
        # Parse JSON response
        data = response.json()
        
        # Append transactions to the list
        transactions = data.get('transactions', [])
        all_results.extend(transactions)
        
        # Check for marker to continue pagination
        marker = data.get('marker')
        if not marker:
            break  # Stop when no more pages are available
        
    # Print the total number of results retrieved
    print(f"Total records fetched: {len(all_results)}")
    return all_results

In [28]:
# base_url = f'https://api.xrpscan.com/api/v1/amm/pools'
# amm_data = get_pagination_results(base_url)

In [29]:
# amm_df = pd.DataFrame(amm_data)

In [30]:
# amm_df["AssetName_Extracted"] = amm_df["AssetName"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
# amm_df["Asset2Name_Extracted"] = amm_df["Asset2Name"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)


In [31]:
# filtered_df = amm_df[(amm_df["AssetName_Extracted"] == "RLUSD") | (amm_df["Asset2Name_Extracted"] == "RLUSD")]
# filtered_df[filtered_df['Balance'] > 0]

In [32]:
# rlusd_pools = filtered_df['Account'].unique()

In [33]:
# all_results = []

# for pool in rlusd_pools:
#     base_url = f'https://api.xrpscan.com/api/v1/amm/{pool}'
#     data = call_api(base_url)
#     all_results.append(data)

In [34]:
# rl_usd_xrp_pool_data = pd.DataFrame(all_results)

In [35]:
base_url = f'https://api.xrpscan.com/api/v1/amm/rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3'
data = call_api(base_url)
rl_usd_xrp_pool_data = pd.DataFrame([data])
rl_usd_xrp_pool_data

data: {'account': 'rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3', 'amount': '666185571998', 'amount2': {'currency': '524C555344000000000000000000000000000000', 'issuer': 'rMxCKbEDwqr76QuheSUMdEGf4B9xJ8m5De', 'value': '1535635.383894134'}, 'asset2_frozen': False, 'auction_slot': {'account': 'rPSvcmNAvc6DAZYX2w5WM4bcnLKSxyt8VY', 'discounted_fee': 23, 'expiration': '2025-01-31T19:53:30+0000', 'price': {'currency': '03B245BE580EC4F4386A751C084489EC4B514A2F', 'issuer': 'rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3', 'value': '0'}, 'time_interval': 20}, 'lp_token': {'currency': '03B245BE580EC4F4386A751C084489EC4B514A2F', 'issuer': 'rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3', 'value': '978747322.1161325'}, 'trading_fee': 234, 'vote_slots': [{'account': 'rwizard1gGgtcnNBb8mJQXDQfPpsVX69bY', 'trading_fee': 300, 'vote_weight': 3785}, {'account': 'rJDRwJ9oSiB5yUDEEdxEy9xBEHR9ekBmh', 'trading_fee': 190, 'vote_weight': 11819}, {'account': 'rKuYdrNLbHM1RYkAnccnRJhKRxNnAMixPA', 'trading_fee': 175, 'vote_weight': 9191}, {'accou

,account,amount,amount2,asset2_frozen,auction_slot,lp_token,trading_fee,vote_slots
0,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,666185571998,{'currency': '524C5553440000000000000000000000...,False,{'account': 'rPSvcmNAvc6DAZYX2w5WM4bcnLKSxyt8V...,{'currency': '03B245BE580EC4F4386A751C084489EC...,234,[{'account': 'rwizard1gGgtcnNBb8mJQXDQfPpsVX69...


In [36]:
rl_usd_xrp_pool_data["amount2_value"] = rl_usd_xrp_pool_data["amount2"].apply(lambda x: float(x["value"]) if isinstance(x, dict) and "value" in x else None)
rl_usd_xrp_pool_data

,account,amount,amount2,asset2_frozen,auction_slot,lp_token,trading_fee,vote_slots,amount2_value
0,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,666185571998,{'currency': '524C5553440000000000000000000000...,False,{'account': 'rPSvcmNAvc6DAZYX2w5WM4bcnLKSxyt8V...,{'currency': '03B245BE580EC4F4386A751C084489EC...,234,[{'account': 'rwizard1gGgtcnNBb8mJQXDQfPpsVX69...,1.535635e+06


In [37]:
rl_usd_xrp_pool_data["token1"] = rl_usd_xrp_pool_data["amount"].apply(lambda x: x["currency"] if isinstance(x, dict) else "XRP")
rl_usd_xrp_pool_data["amount1"] = rl_usd_xrp_pool_data["amount"].apply(lambda x: float(x["value"]) if isinstance(x, dict) and "value" in x else pd.to_numeric(x, errors="coerce"))

# Extract currency from 'amount2'
rl_usd_xrp_pool_data["token2"] = rl_usd_xrp_pool_data["amount2"].apply(lambda x: x["currency"] if isinstance(x, dict) else "XRP")
rl_usd_xrp_pool_data["amount2"] = rl_usd_xrp_pool_data["amount2"].apply(lambda x: float(x["value"]) if isinstance(x, dict) and "value" in x else pd.to_numeric(x, errors="coerce"))


In [38]:
rl_usd_xrp_pool_data["token1"] = rl_usd_xrp_pool_data["token1"].replace("524C555344000000000000000000000000000000", "RLUSD")
rl_usd_xrp_pool_data["token2"] = rl_usd_xrp_pool_data["token2"].replace("524C555344000000000000000000000000000000", "RLUSD")

In [39]:
rl_usd_xrp_pool_data[["account", "token1", "amount1", "token2", "amount2"]]

,account,token1,amount1,token2,amount2
0,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,666185571998,RLUSD,1.535635e+06


In [40]:
xpr_rlusd_pool = rl_usd_xrp_pool_data[
    ((rl_usd_xrp_pool_data["token1"] == "XRP") | (rl_usd_xrp_pool_data["token1"] == "RLUSD")) &
    ((rl_usd_xrp_pool_data["token2"] == "XRP") | (rl_usd_xrp_pool_data["token2"] == "RLUSD"))
]

In [41]:
xpr_rlusd_pool['amount1_norm'] = xpr_rlusd_pool['amount1'] / 1e6

In [42]:
xpr_rlusd_pool['amount1_norm'].values[0]

np.float64(666185.571998)

In [43]:
rlusd_in_xrp_lp = xpr_rlusd_pool['amount2_value'].values[0]

In [44]:
xrp_in_xrp_lp = xpr_rlusd_pool['amount1_norm'].values[0]

In [45]:
xrp_in_xrp_lp

np.float64(666185.571998)

In [46]:
today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

In [47]:
eth_rlusd_pool_query1 = lp_data('0xd001ae433f254283fece51d4acce8c53263aa186')
eth_rlusd_pool = flipside_api_results(eth_rlusd_pool_query1,FLIPSIDE_KEY)

Query not completed. Retrying in 30 seconds...
Query not completed. Retrying in 30 seconds...


In [48]:
eth_rlusd_pool_query2 = lp_data('0xcc6d2f26d363836f85a42d249e145ec0320d3e55')
eth_rlusd_pool2 = flipside_api_results(eth_rlusd_pool_query2,FLIPSIDE_KEY)

Query not completed. Retrying in 30 seconds...


In [49]:
eth_rlusd_pool.dropna(inplace=True)
eth_rlusd_pool2.dropna(inplace=True)

In [ ]:
eth_rlusd_pool = pd.concat([eth_rlusd_pool.drop(columns=['total_tvl','__row_index']),eth_rlusd_pool2.drop(columns=['total_tvl','__row_index'])]).groupby(['symbol','dt']).sum().reset_index()
eth_rlusd_pool['dt'] = pd.to_datetime(eth_rlusd_pool['dt'])
eth_rlusd_pool.set_index('dt',inplace=True)
eth_rlusd_pool.sort_index(inplace=True)
total_tvl = eth_rlusd_pool.groupby(eth_rlusd_pool.index)['tvl'].sum()

eth_rlusd_pool = eth_rlusd_pool.merge(
    total_tvl.to_frame('total_tvl'),
    left_index=True,
    right_index=True,
    how='inner'
)


In [69]:
eth_rlusd_bal_timeseries = eth_rlusd_pool[eth_rlusd_pool['symbol']=='RLUSD']

In [70]:
eth_rlusd_pool.groupby(eth_rlusd_pool.index)[['total_tvl']].last()

,total_tvl
dt,
2025-01-16 14:00:00+00:00,9.873170e+03
2025-01-16 15:00:00+00:00,9.873066e+03
2025-01-16 16:00:00+00:00,9.872010e+03
2025-01-16 17:00:00+00:00,9.889454e+03
2025-01-16 18:00:00+00:00,9.872686e+03
...,...
2025-02-24 18:00:00+00:00,1.057870e+07
2025-02-24 19:00:00+00:00,1.056875e+07
2025-02-24 20:00:00+00:00,1.058732e+07


In [71]:
fig2 = visualization_pipeline(
df=eth_rlusd_pool.groupby(eth_rlusd_pool.index)[['total_tvl']].last(),
    title='rlusd_fig2',
    # start_date='2024-01-01',
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='line',
    dimensions=dict(height=351, width=730),
    # groupby = 'Blockchain',
    # num_col='Supply',
    cols_to_plot=['total_tvl'],
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    axes_data=dict(y1=['total_tvl'],y2=[]),
    # tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=1,
    y2=True,
    # min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=False,
    hole_size=0,
    # line_width=1,
    descending=False,
    buffer = 0.25,
    annotations=True,
    text=True,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=True,
    annotation_prefix="$",
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': ['total_tvl'], 'y2': []}
tick0: 2025-01-16 14:00:00+00:00
turn_to_time: True


In [72]:
chartBuilder(
    fig=fig2,
    add_the_date=False,
    clean_columns=True,
    date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    save=False,
    topn=4,
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
capwords: []
y1_list: ['total_tvl'] 
y2_list: []
'total_tvl' does not match capword. Converting to title case: Total
'total_tvl' does not match capword. Converting to title case: Tvl
Cleaned string: 'Total Tvl'
'total_tvl' does not match capword. Converting to title case: Total
'total_tvl' does not match capword. Converting to title case: Tvl
Cleaned string: 'Total Tvl'
Cleaned DataFrame columns: ['Total Tvl']
Cleaned cols_to_plot: ['Total Tvl']
Cleaned bar_col: []
Cleaned line_col: []
Cleaned y1_list: ['Total Tvl']
Cleaned y2_list: []
Cleaned groupby: None
Cleaned num_col: None
logo path: ../img/Logos/None
Generating line plot...
axes titles at viz pipeline: {'y1': None, 'y2': None}
No GroupBy Col
cols to plot: ['Total Tvl']
axes data to plot: {'y1': ['Total Tvl'], 'y2': []}
tick0 in func: 2025-01-16 14:00:00+00:00
sort_list: True
Ranked columns: Total Tvl    6.364063e+09
dtype: float64
Ranked columns: Total Tvl    1.057624e+07
Name: 2025-02-24 22:00:00+00:00, dtype: float6

e:\projects\chart_builder_pkg\chart_builder\scripts\plots.py:232: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



x:10576242.310014796M
x:9873.169983001K
x:10767830.774566486M
axes_font_colors: {}
ticksy: None
[x_range_start, x_range_end]: [Timestamp('2025-01-16 08:00:00+0000', tz='UTC'), Timestamp('2025-02-25 04:00:00+0000', tz='UTC')]
legend_orientation: v
Columns: Index(['Total Tvl'], dtype='object') 
Index: DatetimeIndex(['2025-01-16 14:00:00+00:00', '2025-01-16 15:00:00+00:00',
               '2025-01-16 16:00:00+00:00', '2025-01-16 17:00:00+00:00',
               '2025-01-16 18:00:00+00:00', '2025-01-16 19:00:00+00:00',
               '2025-01-16 20:00:00+00:00', '2025-01-16 21:00:00+00:00',
               '2025-01-16 22:00:00+00:00', '2025-01-16 23:00:00+00:00',
               ...
               '2025-02-24 13:00:00+00:00', '2025-02-24 14:00:00+00:00',
               '2025-02-24 15:00:00+00:00', '2025-02-24 16:00:00+00:00',
               '2025-02-24 17:00:00+00:00', '2025-02-24 18:00:00+00:00',
               '2025-02-24 19:00:00+00:00', '2025-02-24 20:00:00+00:00',
               '2025-02

In [54]:
fig3 = visualization_pipeline(
df=eth_rlusd_pool,
    title='rlusd_fig3',
    # start_date='2024-01-01',
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='line',
    dimensions=dict(height=351, width=730),
    groupby = 'symbol',
    num_col='tvl',
    # cols_to_plot=['total_tvl'],
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['total_tvl'],y2=[]),
    # tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=1,
    y2=True,
    # min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=True,
    hole_size=0,
    # line_width=1,
    descending=False,
    buffer = 0.25,
    annotations=False,
    text=True,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=False,
    annotation_prefix="$",
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2025-01-16 14:00:00+00:00
turn_to_time: True
df sort order at init: ['RLUSD' 'USDC']


In [ ]:
chartBuilder(
    fig=fig3,
    add_the_date=True,
    clean_columns=True,
    date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    save=False,
    title='RLUSD-USDC Pool on Ethereum',
    topn=4,
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
capwords: []
y1_list: All 
y2_list: []
'symbol' does not match capword. Converting to title case: Symbol
Cleaned string: 'Symbol'
'tvl' does not match capword. Converting to title case: Tvl
Cleaned string: 'Tvl'
'A' does not match capword. Converting to title case: A
Cleaned string: 'A'
'l' does not match capword. Converting to title case: L
Cleaned string: 'L'
'l' does not match capword. Converting to title case: L
Cleaned string: 'L'
'symbol' does not match capword. Converting to title case: Symbol
Cleaned string: 'Symbol'
Cleaned groupby: Symbol
'tvl' does not match capword. Converting to title case: Tvl
Cleaned string: 'Tvl'
Cleaned num_col: Tvl
Cleaned DataFrame columns: ['Symbol', 'Tvl']
Cleaned cols_to_plot: ['Symbol', 'Tvl']
Cleaned bar_col: []
Cleaned line_col: []
Cleaned y1_list: ['A', 'L', 'L']
Cleaned y2_list: []
Cleaned groupby: Symbol
Cleaned num_col: Tvl
logo path: ../img/Logos/None
Generating line plot...
axes titles at viz pipeline: {'y1': None, 'y2': None}


In [74]:
eth_rlusd_pool[eth_rlusd_pool['symbol']=='RLUSD'].iloc[0]

symbol               RLUSD
current_bal    4934.513133
tvl            4932.904482
total_tvl      9873.169983
Name: 2025-01-16 14:00:00+00:00, dtype: object

In [ ]:
rlusd_in_eth_lp = eth_rlusd_pool[eth_rlusd_pool['symbol']=='RLUSD'].iloc[0][['current_bal']].values[0]

In [57]:
rlusd_in_eth_lp

np.float64(4769374.05912583)

In [58]:
rlusd_in_xrp_lp

np.float64(1614553.902805907)

In [59]:
total_rlusd_in_lp = rlusd_in_eth_lp + rlusd_in_xrp_lp

In [60]:
total_rlusd_in_lp

np.float64(6383927.961931737)

In [61]:
percent_rlusd_in_lp = (total_rlusd_in_lp / total_rlusd_supply)*100
f'{percent_rlusd_in_lp:.2f}%'

'5.20%'

In [62]:
supply_comp = {
    "Status":["In-Liquidity","Out-Liquidity"],
    "Amount":[total_rlusd_in_lp, (total_rlusd_supply-total_rlusd_in_lp)]
}

supply_comp_df = pd.DataFrame(supply_comp)

supply_comp_df

,Status,Amount
0,In-Liquidity,6.383928e+06
1,Out-Liquidity,1.164328e+08


In [204]:
fig3 = visualization_pipeline(
df=supply_comp_df,
    title='rlusd_fig3',
    # start_date='2024-01-01',
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='pie',
    dimensions=dict(height=351, width=730),
    groupby = 'Status',
    num_col='Amount',
    # cols_to_plot='All',
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['UAW'],y2=[]),
    tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=0,
    y2=True,
    # min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines+markers',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=False,
    hole_size=0,
    line_width=0,
    descending=False,
    buffer = 0.25,
    annotations=True,
    text=True,
    orientation='h',
    sort_list=True,
    margin=dict(l=0, r=0, t=100, b=0),
    autosize=False,
    max_annotation=True,
    annotation_prefix=None,
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['In-Liquidity' 'Out-Liquidity']


In [206]:
chartBuilder(
    fig=fig3,
    add_the_date=False,
    clean_columns=True,
    date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    save=False,
    title_xy=dict(x=0.1,y=0.9),
    topn=4,
    title='Share of RLUSD Locked in Liquidity Pools'
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
capwords: []
y1_list: ['A', 'L', 'L'] 
y2_list: []
'Amount' does not match capword. Converting to title case: Amount
Cleaned string: 'Amount'
'Status' does not match capword. Converting to title case: Status
Cleaned string: 'Status'
'A' does not match capword. Converting to title case: A
Cleaned string: 'A'
'L' does not match capword. Converting to title case: L
Cleaned string: 'L'
'L' does not match capword. Converting to title case: L
Cleaned string: 'L'
'Status' does not match capword. Converting to title case: Status
Cleaned string: 'Status'
Cleaned groupby: Status
'Amount' does not match capword. Converting to title case: Amount
Cleaned string: 'Amount'
Cleaned num_col: Amount
Cleaned DataFrame columns: ['Amount', 'Status']
Cleaned cols_to_plot: ['Amount', 'Status']
Cleaned bar_col: []
Cleaned line_col: []
Cleaned y1_list: ['A', 'L', 'L']
Cleaned y2_list: []
Cleaned groupby: Status
Cleaned num_col: Amount
logo path: ../img/Logos/None
Generating pie chart...
self.textinf

In [65]:
from diskcache import Cache

In [66]:
cache = Cache('data_collection')

In [67]:
xrpl_lp_timeseries = cache.get('timeseries')

In [68]:
xrpl_lp_timeseries['dt'] = pd.to_datetime(xrpl_lp_timeseries['dt'])
xrpl_lp_timeseries['dt'] = pd.to_datetime(xrpl_lp_timeseries['dt'].dt.strftime('%Y-%m-%d %H:00:00'))

In [69]:
xrpl_lp_timeseries.set_index('dt',inplace=True)

In [70]:
xrpl_lp_timeseries.index

DatetimeIndex(['2025-02-17 21:00:00', '2025-02-17 22:00:00',
               '2025-02-17 23:00:00', '2025-02-18 00:00:00',
               '2025-02-18 01:00:00', '2025-02-18 02:00:00',
               '2025-02-18 03:00:00', '2025-02-18 04:00:00',
               '2025-02-18 05:00:00', '2025-02-18 06:00:00',
               ...
               '2025-02-23 11:00:00', '2025-02-23 12:00:00',
               '2025-02-23 14:00:00', '2025-02-23 15:00:00',
               '2025-02-23 16:00:00', '2025-02-23 17:00:00',
               '2025-02-23 18:00:00', '2025-02-23 19:00:00',
               '2025-02-23 20:00:00', '2025-02-23 21:00:00'],
              dtype='datetime64[ns]', name='dt', length=139, freq=None)

In [71]:
rlusd_xrp_df = xrpl_lp_timeseries[['rlusd_bal']]
rlusd_xrp_df['blockchain'] = 'XRP Ledger'
rlusd_xrp_df.rename(columns={"rlusd_bal":"current_bal"},inplace=True)
rlusd_xrp_df

,current_bal,blockchain
dt,,
2025-02-17 21:00:00,1.614909e+06,XRP Ledger
2025-02-17 22:00:00,1.617292e+06,XRP Ledger
2025-02-17 23:00:00,1.611439e+06,XRP Ledger
2025-02-18 00:00:00,1.610191e+06,XRP Ledger
2025-02-18 01:00:00,1.607343e+06,XRP Ledger
...,...,...
2025-02-23 17:00:00,1.415431e+06,XRP Ledger
2025-02-23 18:00:00,1.418911e+06,XRP Ledger
2025-02-23 19:00:00,1.419123e+06,XRP Ledger


In [72]:
eth_rlusd_bal_timeseries.index = eth_rlusd_bal_timeseries.index.strftime('%Y-%m-%d %H:00:00')

In [73]:
rlusd_eth_df = eth_rlusd_bal_timeseries[['current_bal']]
rlusd_eth_df['blockchain'] = 'Ethereum'
rlusd_eth_df

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_13872\3381202216.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,current_bal,blockchain
dt,,
2025-02-23 20:00:00,4.769374e+06,Ethereum
2025-02-23 19:00:00,4.769374e+06,Ethereum
2025-02-23 18:00:00,4.769374e+06,Ethereum
2025-02-23 17:00:00,4.769374e+06,Ethereum
2025-02-23 16:00:00,4.769374e+06,Ethereum
...,...,...
2025-01-16 18:00:00,4.934513e+03,Ethereum
2025-01-16 17:00:00,4.934513e+03,Ethereum
2025-01-16 16:00:00,4.934513e+03,Ethereum


In [74]:
rlusd_eth_df.index = pd.to_datetime(rlusd_eth_df.index)

In [75]:
rlusd_xrp_df.index

DatetimeIndex(['2025-02-17 21:00:00', '2025-02-17 22:00:00',
               '2025-02-17 23:00:00', '2025-02-18 00:00:00',
               '2025-02-18 01:00:00', '2025-02-18 02:00:00',
               '2025-02-18 03:00:00', '2025-02-18 04:00:00',
               '2025-02-18 05:00:00', '2025-02-18 06:00:00',
               ...
               '2025-02-23 11:00:00', '2025-02-23 12:00:00',
               '2025-02-23 14:00:00', '2025-02-23 15:00:00',
               '2025-02-23 16:00:00', '2025-02-23 17:00:00',
               '2025-02-23 18:00:00', '2025-02-23 19:00:00',
               '2025-02-23 20:00:00', '2025-02-23 21:00:00'],
              dtype='datetime64[ns]', name='dt', length=139, freq=None)

In [76]:
merged_df = pd.merge(
    rlusd_eth_df[['current_bal']].rename(columns={"current_bal":"RLUSD_ETH_LP"}),
    rlusd_xrp_df[['current_bal']].rename(columns={"current_bal":"RLUSD_XRP_LP"}),
    left_index=True,
    right_index=True,
    how='right'
).ffill()

In [77]:
def prepare_data_for_simulation(price_timeseries, start_date, end_date):
    """
    Ensure price_timeseries has entries for start_date and end_date.
    If not, fill in these dates using the last available data.
    """
    # Ensure 'ds' is in datetime format
    # price_timeseries['hour'] = pd.to_datetime(price_timeseries['hour'])
    
    # Set the index to 'ds' for easier manipulation
    # if price_timeseries.index.name != 'hour':
    #     price_timeseries.set_index('hour', inplace=True)

    print(f'price index: {price_timeseries.index}')

    price_timeseries.index = price_timeseries.index.tz_localize(None)
    
    # Check if start_date and end_date exist in the data
    required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
    all_dates = price_timeseries.index.union(required_dates)
    
    # Reindex the dataframe to ensure all dates from start to end are present
    price_timeseries = price_timeseries.reindex(all_dates)
    
    # Forward fill to handle NaN values if any dates were missing
    price_timeseries.fillna(method='ffill', inplace=True)

    # Reset index if necessary or keep the datetime index based on further needs
    # price_timeseries.reset_index(inplace=True, drop=False)
    # price_timeseries.rename(columns={'index': 'hour'}, inplace=True)
    # price_timeseries.set_index('hour',inplace=True)
    
    return price_timeseries

In [78]:
rlusd_eth_df.index

DatetimeIndex(['2025-02-23 20:00:00', '2025-02-23 19:00:00',
               '2025-02-23 18:00:00', '2025-02-23 17:00:00',
               '2025-02-23 16:00:00', '2025-02-23 15:00:00',
               '2025-02-23 14:00:00', '2025-02-23 13:00:00',
               '2025-02-23 12:00:00', '2025-02-23 11:00:00',
               ...
               '2025-01-16 23:00:00', '2025-01-16 22:00:00',
               '2025-01-16 21:00:00', '2025-01-16 20:00:00',
               '2025-01-16 19:00:00', '2025-01-16 18:00:00',
               '2025-01-16 17:00:00', '2025-01-16 16:00:00',
               '2025-01-16 15:00:00', '2025-01-16 14:00:00'],
              dtype='datetime64[ns]', name='dt', length=919, freq=None)

In [79]:
rlusd_eth_df = prepare_data_for_simulation(rlusd_eth_df,rlusd_xrp_df.index.min(),rlusd_xrp_df.index.max())

price index: DatetimeIndex(['2025-02-23 20:00:00', '2025-02-23 19:00:00',
               '2025-02-23 18:00:00', '2025-02-23 17:00:00',
               '2025-02-23 16:00:00', '2025-02-23 15:00:00',
               '2025-02-23 14:00:00', '2025-02-23 13:00:00',
               '2025-02-23 12:00:00', '2025-02-23 11:00:00',
               ...
               '2025-01-16 23:00:00', '2025-01-16 22:00:00',
               '2025-01-16 21:00:00', '2025-01-16 20:00:00',
               '2025-01-16 19:00:00', '2025-01-16 18:00:00',
               '2025-01-16 17:00:00', '2025-01-16 16:00:00',
               '2025-01-16 15:00:00', '2025-01-16 14:00:00'],
              dtype='datetime64[ns]', name='dt', length=919, freq=None)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_13872\1186389551.py:18: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_13872\1186389551.py:25: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [80]:
rlusd_eth_df

,current_bal,blockchain
2025-01-16 14:00:00,4.934513e+03,Ethereum
2025-01-16 15:00:00,4.934513e+03,Ethereum
2025-01-16 16:00:00,4.934513e+03,Ethereum
2025-01-16 17:00:00,4.934513e+03,Ethereum
2025-01-16 18:00:00,4.934513e+03,Ethereum
...,...,...
2025-02-23 17:00:00,4.769374e+06,Ethereum
2025-02-23 18:00:00,4.769374e+06,Ethereum
2025-02-23 19:00:00,4.769374e+06,Ethereum
2025-02-23 20:00:00,4.769374e+06,Ethereum


In [81]:
rlusd_xrp_df.ffill(inplace=True)

In [82]:
daily_eth_lp = rlusd_eth_df.resample('D').last()

In [83]:
daily_xrpl_lp = rlusd_xrp_df.resample('D').last()

In [84]:
combined_rlusd_lp = pd.concat([daily_eth_lp,daily_xrpl_lp])

In [85]:
combined_rlusd_lp

,current_bal,blockchain
2025-01-16,4.617377e+03,Ethereum
2025-01-17,9.287613e+02,Ethereum
2025-01-18,9.140881e+02,Ethereum
2025-01-19,8.146444e+02,Ethereum
2025-01-20,4.978103e+02,Ethereum
2025-01-21,4.356485e+02,Ethereum
2025-01-22,5.377521e+02,Ethereum
2025-01-23,4.180613e+03,Ethereum
2025-01-24,5.278855e+03,Ethereum
2025-01-25,4.212158e+03,Ethereum


In [86]:
combined_rlusd_lp.index = pd.to_datetime(combined_rlusd_lp.index)
combined_rlusd_lp.sort_index(inplace=True)

In [199]:
fig4 = visualization_pipeline(
df=combined_rlusd_lp[combined_rlusd_lp.index >= rlusd_xrp_df.index.min()],
    title='rlusd_fig4',
    start_date=str(rlusd_xrp_df.index.min()),
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='line',
    dimensions=dict(height=351, width=730),
    groupby = 'blockchain',
    num_col='current_bal',
    # cols_to_plot=["RLUSD_ETH_LP","RLUSD_XRP_LP"],
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['total_tvl'],y2=[]),
    # tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=1,
    y2=True,
    min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=True,
    hole_size=0,
    # line_width=1,
    descending=False,
    margin=dict(l=0, r=0, t=100, b=0),
    legend_background=dict(bgcolor='white',bordercolor='black',
                                                                                                    borderwidth=1, itemsizing='constant',
                                                                                                    yanchor="top",xanchor="center",buffer=8),
    # buffer = 0.25,
    annotations=False,
    legend_placement=dict(x=0.75,y=0.9),
    text=True,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=False,
    annotation_prefix="$",
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2025-02-18 00:00:00
turn_to_time: True
df sort order at init: ['Ethereum' 'XRP Ledger']


In [ ]:
chartBuilder(
    fig=fig4,
    add_the_date=True,
    clean_columns=False,
    # date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    title='RLUSD Locked in Liquidity Pools <br>by Chain Over Time',
    title_xy=dict(x=0.1,y=0.9),
    save=False,
    date_xy=dict(x=0.93,y=1.05),
    # title_xy=dict(x=0.1,y=0.9),
    topn=4,
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
logo path: ../img/Logos/None
Generating line plot...
axes titles at viz pipeline: {'y1': None, 'y2': None}
GroupBy Col: blockchain
tick0: 2025-02-18 00:00:00
df @ rank_by_col:              current_bal  blockchain  marker_size
2025-02-18  4.755673e+06    Ethereum            3
2025-02-18  1.394979e+06  XRP Ledger            3
2025-02-19  1.445532e+06  XRP Ledger            3
2025-02-19  4.755673e+06    Ethereum            3
2025-02-20  4.755946e+06    Ethereum            3
2025-02-20  1.425331e+06  XRP Ledger            3
2025-02-21  4.769301e+06    Ethereum            3
2025-02-21  1.411162e+06  XRP Ledger            3
2025-02-22  1.408027e+06  XRP Ledger            3
2025-02-22  4.769315e+06    Ethereum            3
2025-02-23  4.769374e+06    Ethereum            3
2025-02-23  1.418783e+06  XRP Ledger            3
df:              current_bal  blockchain  marker_size
2025-02-18  4.755673e+06    Ethereum            3
2025-02-18  1.394979e+06  XRP Ledger            3
2025-02-1

In [89]:
# fig5 = visualization_pipeline(
# df=merged_df,
#     title='rlusd_fig5',
#     # start_date='2024-01-01',
#     # end_date='2024-12-31',
#     # end_date='2024-10-01',
#     chart_type='line',
#     dimensions=dict(height=351, width=730),
#     # groupby = 'blockchain',
#     # num_col='current_bal',
#     cols_to_plot=["RLUSD_ETH_LP","RLUSD_XRP_LP"],
#     # line_col=['cumulative_freq'],
#     # bar_col=['freg'],
#     tickangle=0,
#     dropna=True,
#     area=True,
#     colors=combined_colors,
#     # axes_data=dict(y1=['total_tvl'],y2=[]),
#     # tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
#     legend_font_size=12,
#     font_family=font_family,
#     cumulative_sort=True,
#     normalize=False,
#     decimals=True,
#     decimal_places=1,
#     y2=True,
#     # min_tick_spacing=50,
#     # buffer=3,
#     barmode='group',
#     mode='lines',
#     # font_size=14,
#     # line_width=3,
#     marker_size=3,
#     tickprefix=dict(y1='$',y2=None),
#     ticksuffix=dict(y1=None,y2=None),
#     axes_titles=dict(y1=None,y2=None),
#     show_legend=True,
#     hole_size=0,
#     # line_width=1,
#     descending=False,
#     # buffer = 0.25,
#     annotations=False,
#     text=True,
#     orientation='h',
#     sort_list=True,
#     autosize=False,
#     max_annotation=False,
#     annotation_prefix="$",
#     auto_title=False,
#     axes_font_colors='auto',
#     to_reverse=False,
#     line_color='black',
#     use_single_color=False,
#     custom_annotation = ['2022-10-31'],
#     # axes_font_colors='auto',
# )

In [90]:
# chartBuilder(
#     fig=fig5,
#     add_the_date=True,
#     clean_columns=False,
#     date_xy=dict(x=0.05,y=1.02),
#     keep_top_n=False,
#     # groupby=True,
#     other=True,
#     save=False,
#     topn=4,
#     # dashed_line=True,
#     # date='2023-04-12',
#     # annotation_text='Staked ETH <br> Withdrawl Activated'
# )

In [91]:
external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

In [92]:
rlusd_eth_dex_stats = dune_api_results(4695750,True,'data/rlusd_eth_dex_stats.csv')

In [93]:
xrpl_vol_base_url = f"https://api.geckoterminal.com/api/v2/networks/xrpl/pools/524C555344000000000000000000000000000000.rMxCKbEDwqr76QuheSUMdEGf4B9xJ8m5De_XRP/ohlcv/day"

In [94]:
data = call_api(xrpl_vol_base_url)

data: {'data': {'id': '921a16ca-7e6e-43f4-b3fd-829c739602a9', 'type': 'ohlcv_request_response', 'attributes': {'ohlcv_list': [[1740268800, 1.00213263, 1.00794498865852, 0.994921979825152, 1.000654172, 3126616.39533692], [1740182400, 0.999420572, 1.00421146247315, 0.994505189997869, 1.00213263, 1309473.6723128], [1740096000, 1.00027450852012, 1.01534651624015, 0.99154439999767, 0.999420572, 3566814.47727784], [1740009600, 0.999596767954252, 1.00640982231723, 0.990719471983288, 1.00027450852012, 2087276.50573632], [1739923200, 0.999730541257816, 1.00693834158677, 0.989368106525186, 0.999596767954252, 3253605.48597424], [1739836800, 1.00035349096386, 1.01407215754238, 0.861225056536475, 0.999730541257816, 3172405.20752181], [1739750400, 1.00095290436866, 1.0161970455571, 0.986277001934072, 1.00035349096386, 2923201.11584502], [1739664000, 0.999345491747336, 1.0102846593426, 0.994568217657868, 1.00095290436866, 1420950.35294398], [1739577600, 1.00066855, 1.00821047244149, 0.99314829973256,

In [1]:
eth_vol_base_url = f"https://api.geckoterminal.com/api/v2/networks/eth/pools/0xd001ae433f254283fece51d4acce8c53263aa186/ohlcv/day"

In [2]:
data2 = call_api(eth_vol_base_url)

NameError: name 'call_api' is not defined

In [ ]:
eth_vol = pd.DataFrame(data2['data']['attributes']['ohlcv_list'])

# Rename columns for clarity (assuming standard OHLCV format)
eth_vol.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

# Convert timestamp to datetime for readability
eth_vol['timestamp'] = pd.to_datetime(eth_vol['timestamp'], unit='s')
eth_vol['volume'].sum()

np.float64(631280.9745206372)

In [139]:
rlusd_eth_dex_stats['volume'].sum()

np.float64(750893.0335480002)

In [99]:
xrpl_vol = pd.DataFrame(data['data']['attributes']['ohlcv_list'])

# Rename columns for clarity (assuming standard OHLCV format)
xrpl_vol.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

# Convert timestamp to datetime for readability
xrpl_vol['timestamp'] = pd.to_datetime(xrpl_vol['timestamp'], unit='s')

In [100]:
xrpl_vol

,timestamp,open,high,low,close,volume
0,2025-02-23,1.002133,1.007945,0.994922,1.000654,3.126616e+06
1,2025-02-22,0.999421,1.004211,0.994505,1.002133,1.309474e+06
2,2025-02-21,1.000275,1.015347,0.991544,0.999421,3.566814e+06
3,2025-02-20,0.999597,1.006410,0.990719,1.000275,2.087277e+06
4,2025-02-19,0.999731,1.006938,0.989368,0.999597,3.253605e+06
5,2025-02-18,1.000353,1.014072,0.861225,0.999731,3.172405e+06
6,2025-02-17,1.000953,1.016197,0.986277,1.000353,2.923201e+06
7,2025-02-16,0.999345,1.010285,0.994568,1.000953,1.420950e+06
8,2025-02-15,1.000669,1.008210,0.993148,0.999345,2.021336e+06
9,2025-02-14,0.994836,1.017060,0.977842,1.000669,5.785919e+06


In [101]:
rlusd_eth_dex_stats['dt'] = pd.to_datetime(rlusd_eth_dex_stats['dt'])
rlusd_eth_dex_stats['dt'] = rlusd_eth_dex_stats['dt'].dt.strftime('%Y-%m-%d')

In [102]:

xrpl_vol.set_index('timestamp',inplace=True)
xrpl_vol.index

DatetimeIndex(['2025-02-23', '2025-02-22', '2025-02-21', '2025-02-20',
               '2025-02-19', '2025-02-18', '2025-02-17', '2025-02-16',
               '2025-02-15', '2025-02-14', '2025-02-13', '2025-02-12',
               '2025-02-11', '2025-02-10', '2025-02-09', '2025-02-08',
               '2025-02-07', '2025-02-06', '2025-02-05', '2025-02-04',
               '2025-02-03', '2025-02-02', '2025-02-01', '2025-01-31',
               '2025-01-30', '2025-01-29', '2025-01-28', '2025-01-27',
               '2025-01-26', '2025-01-25', '2025-01-24', '2025-01-23',
               '2025-01-22', '2025-01-21'],
              dtype='datetime64[ns]', name='timestamp', freq=None)

In [103]:
xrpl_vol['blockchain'] = 'XRPL'

In [104]:
rlusd_eth_dex_stats['blockchain'] = 'Ethereum'
rlusd_eth_dex_stats.rename(columns={"vol":"volume"},inplace=True)
rlusd_eth_dex_stats.set_index('dt',inplace=True)
rlusd_eth_dex_stats.index = pd.to_datetime(rlusd_eth_dex_stats.index)

In [105]:
rlusd_eth_dex_stats

,avg_order,num_orders,volume,blockchain
dt,,,,
2025-02-23,81.828212,15,1227.423182,Ethereum
2025-02-22,68.075258,25,1701.881450,Ethereum
2025-02-21,886.888864,46,40796.887742,Ethereum
2025-02-20,166.483999,39,6492.875964,Ethereum
2025-02-19,901.392684,48,43266.848816,Ethereum
2025-02-18,956.812360,39,39229.306753,Ethereum
2025-02-17,361.378263,49,17707.534876,Ethereum
2025-02-16,141.172223,40,5505.716712,Ethereum
2025-02-15,253.360802,21,5320.576840,Ethereum


In [106]:
xrpl_vol

,open,high,low,close,volume,blockchain
timestamp,,,,,,
2025-02-23,1.002133,1.007945,0.994922,1.000654,3.126616e+06,XRPL
2025-02-22,0.999421,1.004211,0.994505,1.002133,1.309474e+06,XRPL
2025-02-21,1.000275,1.015347,0.991544,0.999421,3.566814e+06,XRPL
2025-02-20,0.999597,1.006410,0.990719,1.000275,2.087277e+06,XRPL
2025-02-19,0.999731,1.006938,0.989368,0.999597,3.253605e+06,XRPL
2025-02-18,1.000353,1.014072,0.861225,0.999731,3.172405e+06,XRPL
2025-02-17,1.000953,1.016197,0.986277,1.000353,2.923201e+06,XRPL
2025-02-16,0.999345,1.010285,0.994568,1.000953,1.420950e+06,XRPL
2025-02-15,1.000669,1.008210,0.993148,0.999345,2.021336e+06,XRPL


In [107]:
filtered_rlusd_eth_dex = rlusd_eth_dex_stats[['blockchain','volume']].resample('W-SUN').agg({
    "blockchain":'last',
    "volume":'sum'
})

In [ ]:
filtered_rlusd_xrpl_dex = xrpl_vol[['blockchain','volume']].resample('W-SUN').agg({
    "blockchain":'last',
    "volume":'sum'
})

In [129]:
xrpl_vol

,open,high,low,close,volume,blockchain
timestamp,,,,,,
2025-02-23,1.002133,1.007945,0.994922,1.000654,3.126616e+06,XRPL
2025-02-22,0.999421,1.004211,0.994505,1.002133,1.309474e+06,XRPL
2025-02-21,1.000275,1.015347,0.991544,0.999421,3.566814e+06,XRPL
2025-02-20,0.999597,1.006410,0.990719,1.000275,2.087277e+06,XRPL
2025-02-19,0.999731,1.006938,0.989368,0.999597,3.253605e+06,XRPL
2025-02-18,1.000353,1.014072,0.861225,0.999731,3.172405e+06,XRPL
2025-02-17,1.000953,1.016197,0.986277,1.000353,2.923201e+06,XRPL
2025-02-16,0.999345,1.010285,0.994568,1.000953,1.420950e+06,XRPL
2025-02-15,1.000669,1.008210,0.993148,0.999345,2.021336e+06,XRPL


In [130]:
filtered_rlusd_xrpl_dex

,blockchain,volume
timestamp,,
2025-01-26,XRPL,7.153195e+06
2025-02-02,XRPL,2.916861e+07
2025-02-09,XRPL,3.452254e+07
2025-02-16,XRPL,2.119616e+07
2025-02-23,XRPL,1.943939e+07


In [109]:
combined_vol = pd.concat([filtered_rlusd_eth_dex, filtered_rlusd_xrpl_dex])

In [110]:
combined_vol.sort_index(inplace=True)

In [132]:
combined_vol_total = combined_vol.groupby(combined_vol.index)[['volume']].sum()
combined_vol_total

,volume
2025-01-19,5.143131e+03
2025-01-26,7.231896e+06
2025-02-02,2.931578e+07
2025-02-09,3.474920e+07
2025-02-16,2.133895e+07
2025-02-23,1.958982e+07


In [112]:
vol_by_chain = combined_vol.groupby('blockchain')[['volume']].sum().reset_index()

In [178]:
fig7 = visualization_pipeline(
df=vol_by_chain,
    title='rlusd_fig7',
    # start_date='2024-01-01',
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='pie',
    dimensions=dict(height=351, width=730),
    groupby = 'blockchain',
    num_col='volume',
    # cols_to_plot='All',
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['UAW'],y2=[]),
    tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=0,
    y2=True,
    # min_tick_spacing=50,
    # buffer=3,
    barmode='group',
    mode='lines+markers',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=False,
    hole_size=0,
    line_width=0,
    descending=False,
    buffer = 0.25,
    margin=dict(l=0, r=0, t=100, b=0),
    annotations=True,
    text=True,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=True,
    annotation_prefix='$',
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['Ethereum' 'XRPL']


In [179]:
chartBuilder(
    fig=fig7,
    add_the_date=False,
    clean_columns=False,
    date_xy=dict(x=0.05,y=1.02),
    keep_top_n=False,
    # groupby=True,
    other=True,
    save=False,
    title_xy=dict(x=0.1,y=0.92),
    title='RLUSD Cumulative DEX Volume <br>by Chain',
    topn=4,
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
logo path: ../img/Logos/None
Generating pie chart...
self.textinfo: percent+label
original_labels: ['Ethereum' 'XRPL']
df_copy:                   volume
blockchain              
Ethereum    7.508930e+05
XRPL        1.114799e+08
percent+label, ['XRPL    ', 'Ethereum    ']
textinfo: percent+label
x:112230780.06720082M
Columns: Index(['blockchain', 'volume'], dtype='object') 
Index: RangeIndex(start=0, stop=2, step=1)


In [155]:
fig6 = visualization_pipeline(
df=combined_vol,
    title='rlusd_fig6',
    start_date=str(rlusd_xrp_df.index.min()),
    # end_date='2024-12-31',
    # end_date='2024-10-01',
    chart_type='bar',
    dimensions=dict(height=351, width=730),
    groupby = 'blockchain',
    num_col='volume',
    # cols_to_plot=["RLUSD_ETH_LP","RLUSD_XRP_LP"],
    # line_col=['cumulative_freq'],
    # bar_col=['freg'],
    tickangle=0,
    dropna=True,
    area=True,
    colors=combined_colors,
    # axes_data=dict(y1=['total_tvl'],y2=[]),
    # tickformat=dict(x='%b<br>`%y',y1=None,y2=None),
    legend_font_size=12,
    font_family=font_family,
    legend_placement=dict(x=0.8,y=0.9),
    cumulative_sort=True,
    normalize=False,
    decimals=True,
    decimal_places=1,
    y2=True,
    min_tick_spacing=50,
    # buffer=3,
    # barmode='group',
    mode='lines',
    # font_size=14,
    # line_width=3,
    marker_size=3,
    margin=dict(l=0, r=0, t=50, b=0),
    tickprefix=dict(y1='$',y2=None),
    ticksuffix=dict(y1=None,y2=None),
    axes_titles=dict(y1=None,y2=None),
    show_legend=True,
    hole_size=0,
    # line_width=1,
    descending=True,
    buffer = 3,
    annotations=False,
    text=False,
    orientation='h',
    sort_list=True,
    autosize=False,
    max_annotation=False,
    annotation_prefix="$",
    auto_title=False,
    axes_font_colors='auto',
    to_reverse=False,
    line_color='black',
    use_single_color=False,
    custom_annotation = ['2022-10-31'],
    # axes_font_colors='auto',
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2025-01-19 00:00:00
turn_to_time: True
df sort order at init: ['Ethereum' 'XRPL']


In [157]:
chartBuilder(
    fig=fig6,
    add_the_date=True,
    clean_columns=False,
    date_xy=dict(x=0.87,y=.98),
    title_xy=dict(x=0.1,y=0.9),
    keep_top_n=False,
    # groupby=True,
    other=True,
    title='RLUSD DEX Volume By Chain',
    save=False,
    topn=4,
    # dashed_line=True,
    # date='2023-04-12',
    # annotation_text='Staked ETH <br> Withdrawl Activated'
)

save:False
logo path: ../img/Logos/None
Generating bar plot...
GroupBy Col: blockchain
legend_orientation: v
cumulative_sort: True
sorted_bar_legend_orientation: v
x_range_start:2025-01-16 00:00:00
df @ rank_by_col:            blockchain        volume
2025-01-19   Ethereum  5.143131e+03
2025-01-26   Ethereum  7.870095e+04
2025-01-26       XRPL  7.153195e+06
2025-02-02   Ethereum  1.471697e+05
2025-02-02       XRPL  2.916861e+07
2025-02-09   Ethereum  2.266607e+05
2025-02-09       XRPL  3.452254e+07
2025-02-16   Ethereum  1.427957e+05
2025-02-16       XRPL  2.119616e+07
2025-02-23   Ethereum  1.504228e+05
2025-02-23       XRPL  1.943939e+07
df:            blockchain        volume
2025-01-19   Ethereum  5.143131e+03
2025-01-26   Ethereum  7.870095e+04
2025-01-26       XRPL  7.153195e+06
2025-02-02   Ethereum  1.471697e+05
2025-02-02       XRPL  2.916861e+07
2025-02-09   Ethereum  2.266607e+05
2025-02-09       XRPL  3.452254e+07
2025-02-16   Ethereum  1.427957e+05
2025-02-16       XRPL  2

In [143]:
# base_url = f'https://api.xrpscan.com/api/v1/account/{RLUSD_XRP}/transactions'
# rlusd_xrpl_tx_data = get_tx_pagination_results(base_url)

In [118]:
def identify_mints_and_burns(transactions, issuer_address):
    mints = []
    burns = []

    for tx in transactions:
        # Check if the transaction is validated and has a successful result
        if tx.get('validated', False) and tx.get('meta', {}).get('TransactionResult') == 'tesSUCCESS':
            
            # Check if it's an OfferCreate transaction
            if tx['TransactionType'] == 'OfferCreate':
                taker_gets = tx.get('TakerGets', {})
                taker_pays = tx.get('TakerPays', {})

                # Check if RLUSD is involved in the transaction
                if isinstance(taker_gets, dict) and taker_gets.get('issuer') == issuer_address:
                    # Burn: Offering RLUSD to receive another currency (e.g., XRP)
                    burns.append({
                        'tx_hash': tx['hash'],
                        'amount': float(taker_gets['value']),
                        'type': 'Burn',
                        'date': tx['date']
                    })

                elif isinstance(taker_pays, dict) and taker_pays.get('issuer') == issuer_address:
                    # Mint: Paying in another currency to receive RLUSD
                    mints.append({
                        'tx_hash': tx['hash'],
                        'amount': float(taker_pays['value']),
                        'type': 'Mint',
                        'date': tx['date']
                    })

    return mints, burns

In [119]:
# transactions = rlusd_xrpl_tx_data
# mints, burns = identify_mints_and_burns(transactions, RLUSD_XRP)


In [120]:
# print("Mints:")
# for mint in mints:
#     print(f"Hash: {mint['tx_hash']} | Amount: {mint['amount']} | Date: {mint['date']}")

# print("\nBurns:")
# for burn in burns:
#     print(f"Hash: {burn['tx_hash']} | Amount: {burn['amount']} | Date: {burn['date']}")

In [121]:
external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

In [122]:
app.layout = html.Div(style={'backgroundColor': 'var(--color-background)'}, children=[
    html.H1(
        children='RLUSD Dashboard',
        style={
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)',
            'fontSize': '36px',
            'fontWeight': 'bold',
            'marginBottom': '20px'
        }
    ),
    dcc.Graph(id='supply_by_chain', figure=fig1.return_fig()),
    dcc.Graph(id='ETH_LP_TVL', figure=fig2.return_fig()),
    dcc.Graph(id='supply_by_liquidity', figure=fig3.return_fig()),
    dcc.Graph(id='supply_by_liquidity2', figure=fig4.return_fig())
])

if __name__ == '__main__':
    app.run_server(debug=True)